假设pytorch是一个工具箱
dir()函数，让我们知道工具箱以及工具箱中的分隔区有什么东西
help()函数，让我们知道每个工具如何使用，工具的使用方法

In [6]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"     #判断当前cuda是否可用，可用则在gpu运行，不可用则默认在cpu运行
print(f"Using {device} device")

Using cuda device


In [7]:
class NeuralNetwork(nn.Module):    #声明父类
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()    #创建flatten层
        self.linear_relu_stack = nn.Sequential(      #stack层
            nn.Linear(28*28, 512),   #线性层
            nn.ReLU(),               #非线性函数
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):     #前向运算
        x = self.flatten(x)   #维度展开
        logits = self.linear_relu_stack(x)
        return logits    #在pytorch中Linear层的输出就是logits 另一种理解的角度就是能量
    # 得到logits之后讲过softmax 然后就可以得到预测的概率 之后就可以进行计算损失

In [8]:
model = NeuralNetwork().to(device)   #实例化一个网络，并将网络传入一个设备之中
print(model)   #输出方法一：print输出

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [16]:
from torchsummary import summary
summary(model, input_size=(1, 28, 28))     #输出方式二：第三方库 pytorch summary

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 512]         401,920
              ReLU-3                  [-1, 512]               0
            Linear-4                  [-1, 512]         262,656
              ReLU-5                  [-1, 512]               0
            Linear-6                   [-1, 10]           5,130
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.55
Estimated Total Size (MB): 2.58
----------------------------------------------------------------


In [17]:
X = torch.rand(1, 28, 28, device=device)    #在device随机初始化一个1*28*28的张量
logits = model(X)
#预测概率
pred_probab = nn.Softmax(dim=1)(logits)   #实例化Softmax层，按第一维传入logits
y_pred = pred_probab.argmax(1)          #对概率算出最大值， 利用argmax得到最大的一个概率值的索引
print(f"Predicted class: {y_pred}")     #得到 y预测的标签的索引

Predicted class: tensor([1], device='cuda:0')


In [19]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [20]:
flatten = nn.Flatten()    #从第一维度到最后一个维度浓缩成一个维度
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [21]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [22]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.3217, -0.6308, -0.1737,  0.0678, -0.1291,  0.5421, -0.0449,  0.1629,
          0.0534, -0.0105, -0.1587, -0.3714, -1.0072,  0.2410,  0.0453, -0.1909,
         -0.2916, -0.2430, -0.1410,  0.3978],
        [-0.3950, -0.4006, -0.0053, -0.1947, -0.0989,  0.2439, -0.0234,  0.1176,
          0.2682,  0.0267, -0.2111, -0.2076, -0.9404,  0.3137, -0.2214,  0.2280,
         -0.2514, -0.2207,  0.0856,  0.5380],
        [-0.0503, -0.5505, -0.0323, -0.3838,  0.0600,  0.3801,  0.1039,  0.4847,
          0.4005,  0.0433, -0.4491, -0.3570, -1.2279,  0.2916, -0.1641,  0.3176,
         -0.4015, -0.1948, -0.0861,  0.5386]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0678, 0.0000, 0.5421, 0.0000, 0.1629, 0.0534,
         0.0000, 0.0000, 0.0000, 0.0000, 0.2410, 0.0453, 0.0000, 0.0000, 0.0000,
         0.0000, 0.3978],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2439, 0.0000, 0.1176, 0.2682,
         0.0267, 0.0000, 0.0000, 0.0000, 0.3137, 0.00

In [23]:
seq_modules = nn.Sequential(     #有序化容器
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)    

In [24]:
softmax = nn.Softmax(dim=1)     #指数后的归一化
pred_probab = softmax(logits)   #分类后的概率分布

In [25]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0056, -0.0132, -0.0171,  ..., -0.0062,  0.0247, -0.0037],
        [-0.0194,  0.0144, -0.0067,  ...,  0.0232, -0.0068, -0.0192]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0282, 0.0053], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0065, -0.0307, -0.0371,  ..., -0.0239, -0.0205, -0.0175],
        [-0.0340,  0.0416, -0.0196,  ..., -0.0435, -0.0421, -0.0035]],
       device='cuda:0', grad_fn=<Slic